### Import Library Dependencies

In [ ]:
import zipfile
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import preprocessing
from sklearn.svm import SVC

# Import library Model
from sklearn.neighbors import KNeighborsClassifier

from sklearn.neighbors import NearestNeighbors
from sklearn.metrics.pairwise import cosine_similarity

# Model Klasifikasi Pertama

## Data Loading

Mendownload dataset berupa csv file melalui repository github

Dataset yang digunakan berupa informasi mengenai estimasi kelompok harga dari beberapa smartphone atau mobile devices berdasarkan spesifikasi yang dimiliki perangkat tersebut, seperti RAM, ROM, dan lainnya. Dataset diperoleh melalui proses scraping dari database mobile phone. Dataset:
[Mobile Price Classification](https://github.com/danielrymeds/dataset1/files/7698891/smartphone_specs.csv) 

### Mendownload dan menyiapkan dataset

In [ ]:
 !wget --no-check-certificate \
  https://github.com/danielrymeds/dataset1/files/7712532/mobile-price-classification.zip \

In [ ]:
!wget --no-check-certificate \
  https://github.com/danielrymeds/dataset1/files/7712533/smartphone_specs.csv \

Mengekstrak zip file dari dataset mobile price

In [ ]:
local_zip = '/content/mobile-price-classification.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

Mengubah dataset menjadi dataframe

In [ ]:
df = pd.read_csv('/content/train.csv')
df.head()

## Data Understanding

Melihat informasi dataset secara keseluruhan dan juga secara statistik

In [ ]:
df.info()

In [ ]:
df.describe()

Melihat missing values pada dataframe

In [ ]:
df.isnull().sum()

Memeriksa rasionalitas beberapa variabel

In [ ]:
px_h = (df.px_height==0).sum()
px_w = (df.px_width==0).sum()
sc_h = (df.sc_h==0).sum()
sc_w = (df.sc_w==0).sum()

In [ ]:
print(px_h)
print(px_w)
print(sc_h)
print(sc_w)

Menghapus nilai yang tidak rasional pada variabel

In [ ]:
df = df.loc[(df[['px_height','sc_w']]!=0).all(axis=1)]
df.shape

### Outliers

Melakukan drop outliers pada data

In [ ]:
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR=Q3-Q1
df=df[~((df<(Q1-1.5*IQR))|(df>(Q3+1.5*IQR))).any(axis=1)]
 
# Cek ukuran dataset setelah outliers didrop
df.shape

### Visualisasi Data

Melakukan proses analisis data dengan teknik Univariate EDA

In [ ]:
%matplotlib inline

df.hist(bins=50, figsize=(20,15))
plt.show()

Membuat dataframe baru yang berisi beberapa kolom data dari dataframe awal, untuk nantinya digabungkan dengan data-data hasil scraping

In [ ]:
df_new = pd.DataFrame()

In [ ]:
df_new['Weight(g)'] = df['mobile_wt'].copy()
df_new['Width(px)'] = df['px_width'].copy()
df_new['Height(px)'] = df['px_height'].copy()
df_new['ROM(MB)'] = df['int_memory'].copy()
df_new['RAM(MB)'] = df['ram'].copy()
df_new['Battery Capacity(mAh)'] = df['battery_power'].copy()
df_new['Price'] = df['price_range'].copy()

In [ ]:
df_new

### Pengolahan Dataset Scraping

Membuat dataframe tambahan dari file csv dataset hasil scraping

In [ ]:
df_add = pd.read_csv('/content/smartphone_specs.csv')
df_add

Mengolah Dataset hasil scraping

In [ ]:
df_add = df_add.rename(columns={'Price(USD)': 'Price'})

In [ ]:
df_add.isnull().sum()

In [ ]:
df_add = df_add.dropna()

In [ ]:
df_add.isnull().sum()

In [ ]:
df_add.describe()

Mengambil data dengan nilai weight kurang dari 200g agar hanya termasuk data mobile phone dan bukan data ipad atau tab yang memiliki berat lebih dari 200g

In [ ]:
df_add = df_add.loc[df_add['Weight(g)']<=200]

In [ ]:
df_add

Membagi kolom price menjadi 4 kelas berdasarkan range harga berikut:

Price:


*   Range 0 = < 100 USD
*   Range 1 = 100 - 200 USD
*   Range 2 = 200 - 400 USD
*   Range 3 = >= 400 USD


In [ ]:
df_add.loc[df_add['Price']<100, 'Price'] = 0
df_add.loc[(df_add['Price']>=100) & (df_add['Price']<200), 'Price'] = 1
df_add.loc[(df_add['Price']>=200) & (df_add['Price']<400), 'Price'] = 2
df_add.loc[df_add['Price']>=400, 'Price'] = 3

In [ ]:
df_add.describe()

In [ ]:
df_add

In [ ]:
df_add.Brand.unique()

Hanya mengambil data dengan brand ternama dan pemilik data terbanyak seperti samsung, oppo, vivo, xiaomi, dan realme untuk digabungkan dengan dataset awal

In [ ]:
samsung = df_add.loc[df_add['Brand']=='samsung']
samsung = samsung.drop(columns=['Brand','Model Name','Model Image'])

In [ ]:
oppo = df_add.loc[df_add['Brand']=='oppo']
oppo = oppo.drop(columns=['Brand','Model Name','Model Image'])

In [ ]:
vivo = df_add.loc[df_add['Brand']=='vivo']
vivo = vivo.drop(columns=['Brand','Model Name','Model Image'])

In [ ]:
xiaomi = df_add.loc[df_add['Brand']=='xiaomi']
xiaomi = xiaomi.drop(columns=['Brand','Model Name','Model Image'])

In [ ]:
realme = df_add.loc[df_add['Brand']=='realme']
realme = realme.drop(columns=['Brand','Model Name','Model Image'])

In [ ]:
print(len(df_new))
print(len(samsung))
print(len(oppo))
print(len(vivo))
print(len(xiaomi))
print(len(realme))

### Menggabungkan kedua dataset

In [ ]:
df_final = pd.concat([df_new, samsung, vivo, oppo, xiaomi, realme], ignore_index=True, sort=False)

In [ ]:
df_final

In [ ]:
df_final.isnull().sum()

In [ ]:
df_final.info()

In [ ]:
df_final.describe()

Menghapus nilai yang tidak rasional pada variabel

In [ ]:
df_final = df_final.loc[(df_final['Height(px)']!=1)]
df_final.shape

In [ ]:
df_final.describe()

In [ ]:
df_final

### Pengolahan Outliers dataframe final

In [ ]:
sns.boxplot(x=df_final['Weight(g)'])

In [ ]:
sns.boxplot(x=df_final['Width(px)'])

In [ ]:
sns.boxplot(x=df_final['Height(px)'])

In [ ]:
sns.boxplot(x=df_final['ROM(MB)'])

In [ ]:
sns.boxplot(x=df_final['RAM(MB)'])

In [ ]:
sns.boxplot(x=df_final['Battery Capacity(mAh)'])

In [ ]:
# Proses drop outliers
Q1 = df_final.quantile(0.25)
Q3 = df_final.quantile(0.75)
IQR=Q3-Q1
df_final=df_final[~((df_final<(Q1-1.5*IQR))|(df_final>(Q3+1.5*IQR))).any(axis=1)]
 
# Cek ukuran dataset setelah outliers didrop
df_final.shape

Visualisasi dataframe final

In [ ]:
%matplotlib inline

df_final.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
list(df_final.columns.values[:6])

### Correlation Matrix

Melihat korelasi antar data pada keseluruhan dataset

In [ ]:
plt.figure(figsize=(10, 8))
correlation_matrix = df_final.corr().round(2)

sns.heatmap(data=correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, )
plt.title("Correlation Matrix untuk Fitur Numerik ", size=20)

## Data Preparation

### Train-Test Split

Membagi data menjadi train dan test set dengan ratio 80:20

In [ ]:
y = df_final['Price'] 

li = list(df_final.columns.values[:6])
X = df_final[li]
# Split dataset menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
print(X_test.shape)
print(X_train.shape)

In [ ]:
X

In [ ]:
y

## Modeling

Pada proyek ini digunakan model KNN, dikarenakan setelah ditelusuri, algoritma yang menghasilkan nilai akurasi terbaik adalah model KNN

In [ ]:
# Model KNN
knn = KNeighborsClassifier(n_neighbors=9)

knn.fit(X_train, y_train)

y_pred_knn = knn.predict(X_test)

a_KNN = metrics.accuracy_score(y_test, y_pred_knn)
print("Accuracy:",a_KNN)

Menampilkan metrik presisi, recall, dan juga f1 score dari hasil prediksi model KNN

In [ ]:
print(classification_report(y_test, y_pred_knn))

## Data Testing

Melakukan data testing dengan menggunakan data inputan dari user dan menghasilkan nilai kelompok harga yang sesuai dengan harga di pasaran (USD) dan juga pada dataset

Nilai inputan dari user akan disimpan beserta dengan hasil prediksinya, yang kemudian akan digunakan pada model klasifikasi selanjutnya untuk mendapatkan model mobile phone yang sesuai dengan spesifikasi yang user mau.

In [ ]:
user_input = input()
prediction = knn.predict(np.array([[int(i) for i in user_input.split()]]))

In [ ]:
user_input1 = np.array([int(i) for i in user_input.split()])

In [ ]:
user_input1

In [ ]:
prediction[0]

# Model Rekomendasi

## Data Loading

Menampilkan Mobile phone yang memiliki kelas harga sesuai prediksi

In [ ]:
samsung1 = df_add.loc[df_add['Brand']=='samsung']
oppo1 = df_add.loc[df_add['Brand']=='oppo']
vivo1 = df_add.loc[df_add['Brand']=='vivo']
xiaomi1 = df_add.loc[df_add['Brand']=='xiaomi']
realme1 = df_add.loc[df_add['Brand']=='realme']

In [ ]:
df_rec = pd.concat([samsung1, vivo1, oppo1, xiaomi1, realme1], ignore_index=True, sort=False)
df_rec

In [ ]:
df_pred = df_rec[df_rec['Price'] == prediction[0]]
df_pred

Meminta input user berdasarkan brand mobile phone

In [ ]:
brand_input = input()
df_pred1 = df_pred[df_pred['Brand'] == brand_input]
df_pred1

Membuat dataframe baru dengan satu baris yang berisi data inputan user

In [ ]:
input_data = [['Input', 'Input', np.nan, user_input1[0], user_input1[1], user_input1[2], user_input1[3], user_input1[4], user_input1[4], prediction[0]]]
input_row = pd.DataFrame(input_data, columns=['Brand','Model Name','Model Image', 'Weight(g)', 'Width(px)', 'Height(px)', 'ROM(MB)', 'RAM(MB)', 'Battery Capacity(mAh)', 'Price'])
input_row.index = [700]
input_row

Menggabungkan dataframe input user dengan dataframe rekomendasi

In [ ]:
df_pred2 = pd.concat([df_pred1, input_row], ignore_index=False, sort=False)
df_pred2

In [ ]:
model_name = pd.DataFrame({'Model Name':df_pred2.index})
model_name

## Data Preparation

### Numerical Scaling

Melakukan scaling pada data-data numerik seperti pada kolom Weight(g),	Width(px),	Height(px),	ROM(MB),	RAM(MB), dan Battery Capacity(mAh). Metode ini digunakan untuk menormalkan jangkauan variabel independen atau fitur data. Dalam pemrosesan data, ini juga dikenal sebagai normalisasi data.

In [ ]:
numerical_features = ['Weight(g)', 'Width(px)', 'Height(px)', 'ROM(MB)', 'RAM(MB)', 'Battery Capacity(mAh)']
scaler = MinMaxScaler()
scaler.fit(df_pred2[numerical_features])

In [ ]:
df_pred3 = df_pred2.copy()
df_pred3[numerical_features] = scaler.transform(df_pred3.loc[:, numerical_features])
df_pred3[numerical_features]

In [ ]:
df_pred3

## Modeling

Persiapan data yang akan dimodelkan dengan cara drop kolom yang tidak diperlukan

In [ ]:
df_model = df_pred3.drop(["Brand"],axis = 1)

In [ ]:
df_model = df_model.drop(["Model Name"],axis = 1)
df_model = df_model.drop(["Model Image"],axis = 1)
df_model = df_model.drop(["Price"],axis = 1)
df_model

Menggunakan model Nearest neighbor untuk sistem rekomendasi

In [ ]:
model = NearestNeighbors(metric='euclidean')
model.fit(df_model)

Membuat model dan fungsi yang bertujuan untuk memberikan rekomendasi aplikasi dari contoh aplikasi yang diinputkan.

In [ ]:
def getRecommendedPhone(modelname,recommend_phone=3,get_similarity=False):
    # Memberikan banyaknya rekomendasi mobile phone sesuai dengan nilai banyaknya 'recommend_phone'
    distances,neighbors = model.kneighbors(df_model.loc[modelname],n_neighbors=recommend_phone+1)
    print(f'Similar Phone for "{modelname[0]}:"')
    # Menampilkan nama model phone yang memiliki jarak terdekat dari mobile phone yang diinputkan user
    print(neighbors[0][1:])
    similar_phone = []
    for neighbor in neighbors[0][1:]:
        similar_phone.append(model_name.loc[neighbor][0])
    if not get_similarity:
        return similar_phone
    # Menerapkan nilai similarity dari seluruh mobile phone serupa yang telah melewati proses NearestNeighbors dengan cosine similarity dan mengalikannya dengan 100 agar nilainya berupa persentase
    similarity = []
    for phone in similar_phone:
        sim = cosine_similarity(df_model.loc[[modelname[0]]],df_model.loc[[phone]]).flatten()[0]
        similarity.append(sim*100)
    # Membuat dataframe baru yang berisi mobile phone yang direkomendasikan
    similar_df = pd.DataFrame({'Model':similar_phone,'Similarity':similarity})
    similar_df.sort_values(by='Similarity',ascending=False)
    return similar_df

## Model Testing

Menampilkan 3 nama model mobile phone yang terdekat dengan inputan user (700)

In [ ]:
recommendation = getRecommendedPhone(model_name.iloc[-1],get_similarity=True)
recommendation

Menampilkan Nama model mobile phone yang di rekomendasikan

In [ ]:
index = 0
for row in recommendation['Model']:
  if recommendation['Similarity'][index] > 50:
    print(df_pred2['Model Name'][row])
  else:
    print('Hasil tidak mendekati spesifikasi anda')
  index = index + 1